In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from termcolor import colored
import requests
import io
import time
import os
import json
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from functools import lru_cache

class StockAnalyzer:
    def __init__(self):
        self.base_dir = 'stock_analysis'
        self.stock_lists_dir = os.path.join(self.base_dir, 'stock_lists')
        self.results_dir = os.path.join(self.base_dir, 'daily_results')
        self.delisted_cache_file = os.path.join(self.stock_lists_dir, 'delisted_stocks.json')
        self.setup_directories()
        self.load_delisted_stocks()
        self.session = requests.Session()


    def setup_directories(self):
        """Create necessary directories if they don't exist"""
        os.makedirs(self.stock_lists_dir, exist_ok=True)
        os.makedirs(self.results_dir, exist_ok=True)
        
    def should_update_stock_list(self, exchange):
        """Check if stock list needs updating (weekly update)"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        if not os.path.exists(list_file):
            return True
        # Check if file is older than 7 days
        file_time = datetime.fromtimestamp(os.path.getmtime(list_file))
        return (datetime.now() - file_time).days >= 7
    
    def load_delisted_stocks(self):
        """Load previously identified delisted stocks"""
        try:
            with open(self.delisted_cache_file, 'r') as f:
                self.delisted_stocks = json.load(f)
        except FileNotFoundError:
            self.delisted_stocks = {'NSE': [], 'BSE': []}  # Changed from set to list
            self.save_delisted_stocks()

    def save_delisted_stocks(self):
        """Save delisted stocks to cache"""
        with open(self.delisted_cache_file, 'w') as f:
            json.dump({
                'NSE': self.delisted_stocks['NSE'],
                'BSE': self.delisted_stocks['BSE']
            }, f)
    
    def save_stock_list(self, symbols, exchange):
        """Save stock list to JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        data = {
            'symbols': symbols,
            'last_updated': datetime.now().strftime('%Y-%m-%d')
        }
        with open(list_file, 'w') as f:
            json.dump(data, f)

    def load_stock_list(self, exchange):
        """Load stock list from JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        with open(list_file, 'r') as f:
            data = json.load(f)
        return data['symbols']
    
    def download_nse_stocks(self):
        """Download NSE stock list if needed"""
        if self.should_update_stock_list('NSE'):
            try:
                url = "https://archives.nseindia.com/content/equities/EQUITY_L.csv"
                response = requests.get(url)
                df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
                symbols = df['SYMBOL'].tolist()
                self.save_stock_list(symbols, 'NSE')
                print("NSE stock list updated")
                return symbols
            except Exception as e:
                print(f"Error downloading NSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'NSE_stocks.json')):
                    return self.load_stock_list('NSE')
                return []
        return self.load_stock_list('NSE')

    def download_bse_stocks(self):
        """Download BSE stock list if needed"""
        if self.should_update_stock_list('BSE'):
            try:
                # Using the BSE list from static file or an appropriate source
                url = "https://www.bseindia.com/corporates/List_Scrips.aspx"
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser')
                table = soup.find('table', {'id': 'ContentPlaceHolder1_tblData'})
                symbols = []

                if table:
                    rows = table.find_all('tr')[1:]  # Skip header row
                    for row in rows:
                        cols = row.find_all('td')
                        if cols:
                            bse_code = cols[0].text.strip()
                            symbols.append(bse_code)

                if symbols:
                    self.save_stock_list(symbols, 'BSE')
                    print("BSE stock list updated")
                    return symbols
            except Exception as e:
                print(f"Error downloading BSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'BSE_stocks.json')):
                    return self.load_stock_list('BSE')
                return []
        return self.load_stock_list('BSE')

    @lru_cache(maxsize=1000)
    def calculate_rsi(self, prices_tuple, periods=14):
        """Calculate RSI with caching for performance"""
        prices = pd.Series(prices_tuple)
        delta = prices.diff()
        gain = delta.where(delta > 0, 0).rolling(window=periods).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
        rs = gain / loss
        return float(100 - (100 / (1 + rs)).iloc[-1])

    def calculate_ema(self, data, length, source='Close', offset=0, smoothing_length=9):
        """
        Calculate Exponential Moving Average with specified parameters
        
        Args:
            data (pd.DataFrame): Historical price data
            length (int): Length of EMA period
            source (str): Source column for calculation
            offset (int): Offset for calculation
            smoothing_length (int): Length of initial SMA smoothing
            
        Returns:
            pd.Series: EMA values
        """
        if isinstance(data, pd.Series):
            series_data = data
        else:
            series_data = data[source]
            
        # First calculate SMA as the smoothing line
        sma = series_data.rolling(window=smoothing_length).mean()
        
        # Calculate the multiplier
        multiplier = 2 / (length + 1)
        
        # Initialize EMA with SMA
        ema = pd.Series(index=series_data.index, dtype=float)
        ema.iloc[:length-1] = np.nan
        ema.iloc[length-1] = sma.iloc[length-1]
        
        # Calculate EMA
        for i in range(length, len(series_data)):
            ema.iloc[i] = (series_data.iloc[i] - ema.iloc[i-1]) * multiplier + ema.iloc[i-1]
        
        # Apply offset if specified
        if offset != 0:
            ema = ema.shift(offset)
            
        return ema

    def calculate_tema(self, data, length, source='Close'):
        """
        Calculate the Triple Exponential Moving Average (TEMA) for the given data.
        
        Formula: TEMA = (3 * EMA1) - (3 * EMA2) + EMA3
        Where:
            EMA1 = EMA(price, length)
            EMA2 = EMA(EMA1, length)
            EMA3 = EMA(EMA2, length)
        """
        # Get price series
        if isinstance(data, pd.Series):
            prices = data
        else:
            prices = data[source]
        
        # Calculate multiplier
        multiplier = 2 / (length + 1)
        
        # Calculate EMA1
        ema1 = prices.copy()
        for i in range(1, len(prices)):
            ema1.iloc[i] = (prices.iloc[i] * multiplier) + (ema1.iloc[i-1] * (1 - multiplier))
        
        # Calculate EMA2
        ema2 = ema1.copy()
        for i in range(1, len(ema1)):
            ema2.iloc[i] = (ema1.iloc[i] * multiplier) + (ema2.iloc[i-1] * (1 - multiplier))
        
        # Calculate EMA3
        ema3 = ema2.copy()
        for i in range(1, len(ema2)):
            ema3.iloc[i] = (ema2.iloc[i] * multiplier) + (ema3.iloc[i-1] * (1 - multiplier))
        
        # Calculate TEMA
        tema = (3 * ema1) - (3 * ema2) + ema3
        
        # Replace first 'length' periods with NaN as they won't have enough data
        tema.iloc[:length-1] = np.nan
        
        return tema

    def get_stock_data(self, symbol, exchange='NSE', days=200):
        """Fetch detailed stock data with additional information"""
        if symbol in self.delisted_stocks[exchange]:
            return None, None, None, None

        try:
            time.sleep(1)  # Add a delay of 1 second between requests
            ticker_symbol = f"{symbol}.NS" if exchange == 'NSE' else f"{symbol}.BO"
            ticker = yf.Ticker(ticker_symbol)
            
            # Get stock info
            info = ticker.info
            
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days)

            hist_data = ticker.history(
                start=start_date,
                end=end_date,
                interval="1d"
            )

            if hist_data.empty:
                return None, None, None, None

            # Calculate EMAs
            if len(hist_data) >= 9:
                hist_data['EMA_9'] = self.calculate_ema(hist_data, length=9, source='Close')
            else:
                hist_data['EMA_9'] = float('nan')
                
            if len(hist_data) >= 21:
                hist_data['EMA_21'] = self.calculate_ema(hist_data, length=21, source='Close')
            else:
                hist_data['EMA_21'] = float('nan')
                
            if len(hist_data) >= 50:
                try:
                    hist_data['EMA_50'] = self.calculate_ema(hist_data, length=50, source='Close')
                    hist_data['TEMA_50'] = self.calculate_tema(hist_data, length=50, source='Close')
                    # Add debug print
                    print(f"Debug - {symbol} TEMA calculation successful")
                    print(f"Last TEMA value: {hist_data['TEMA_50'].iloc[-1]}")
                except Exception as e:
                    print(f"Error calculating TEMA for {symbol}: {e}")
                    hist_data['EMA_50'] = float('nan')
                    hist_data['TEMA_50'] = float('nan')
            else:
                hist_data['EMA_50'] = float('nan')
                hist_data['TEMA_50'] = float('nan')
            
            latest_price = hist_data['Close'].iloc[-1]
            prices_tuple = tuple(hist_data['Close'].values)
            latest_rsi = self.calculate_rsi(prices_tuple)

            return latest_price, hist_data, latest_rsi, info

        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            return None, None, None, None
        
    def process_stock_batch(self, batch_data):
        """Process a batch of stocks with enhanced information"""
        symbols, exchange = batch_data
        results = []
        
        with ThreadPoolExecutor(max_workers=8) as executor:
            futures = {executor.submit(self.get_stock_data, symbol, exchange): symbol 
                      for symbol in symbols}
            
            for future in futures:
                symbol = futures[future]
                try:
                    latest_price, hist_data, latest_rsi, info = future.result()
                    if all(v is not None for v in [latest_price, hist_data, latest_rsi, info]):
                        # Extract additional information safely using get() method
                        stock_info = {
                            'Symbol': symbol,
                            'Exchange': exchange,
                            'Company_Name': info.get('longName', 'N/A'),
                            'Latest_Price': latest_price,
                            'EMA_9': hist_data['EMA_9'].iloc[-1] if 'EMA_9' in hist_data else 'N/A',
                            'EMA_21': hist_data['EMA_21'].iloc[-1] if 'EMA_21' in hist_data else 'N/A',
                            'EMA_50': hist_data['EMA_50'].iloc[-1] if 'EMA_50' in hist_data else 'N/A',
                            'TEMA_50': hist_data['TEMA_50'].iloc[-1] if 'TEMA_50' in hist_data else 'N/A',
                            'RSI': latest_rsi,
                            'Volume': hist_data['Volume'].iloc[-1] if 'Volume' in hist_data else 0,
                            'Market_Cap': info.get('marketCap', 'N/A'),
                            'PE_Ratio': info.get('trailingPE', 'N/A'),
                            'EPS': info.get('trailingEps', 'N/A'),
                            'Dividend_Yield': info.get('dividendYield', 'N/A'),
                            'Book_Value': info.get('bookValue', 'N/A'),
                            'Sector': info.get('sector', 'N/A'),
                            'Industry': info.get('industry', 'N/A'),
                            '52W_High': info.get('fiftyTwoWeekHigh', 'N/A'),
                            '52W_Low': info.get('fiftyTwoWeekLow', 'N/A'),
                            '50d_MA': info.get('fiftyDayAverage', 'N/A'),
                            '200d_MA': info.get('twoHundredDayAverage', 'N/A'),
                            'Beta': info.get('beta', 'N/A'),
                            'Previous_Close': info.get('previousClose', 'N/A'),
                            'Open': info.get('open', 'N/A'),
                            'Day_High': info.get('dayHigh', 'N/A'),
                            'Day_Low': info.get('dayLow', 'N/A'),
                            'Date': datetime.now().strftime('%Y-%m-%d')
                        }
                        
                        # Calculate additional metrics
                        if hist_data is not None:
                            stock_info.update({
                                'Volatility_30d': hist_data['Close'].pct_change().std() * np.sqrt(252),
                                'Returns_30d': (hist_data['Close'].iloc[-1] / hist_data['Close'].iloc[0] - 1) * 100,
                                'Average_Volume_30d': hist_data['Volume'].mean()
                            })
                        
                        results.append(stock_info)
                        
                except Exception as e:
                    print(f"Error processing {symbol}: {e}")

        return results


    def run_daily_analysis(self):
        """Run daily analysis with Excel output"""
        current_date = datetime.now().strftime('%Y%m%d')
        
        print("Loading stock lists...")
        nse_symbols = self.download_nse_stocks()
        bse_symbols = self.download_bse_stocks()

        print(f"\nAnalyzing {len(nse_symbols)} NSE and {len(bse_symbols)} BSE stocks")

        for exchange, symbols in [('NSE', nse_symbols), ('BSE', bse_symbols)]:
            batch_data=(symbols, exchange)
            results = self.process_stock_batch(batch_data)
            
            if results:
                # Create Excel writer with xlsxwriter engine
                excel_file = os.path.join(
                    self.results_dir,
                    f'{exchange.lower()}_analysis_{current_date}.xlsx'
                )
                
                # Convert results list to DataFrame
                results_df = pd.DataFrame(results)
                
                with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
                    # Write all stocks to first sheet
                    results_df.to_excel(writer, sheet_name='All Stocks', index=False)
                    
                    # Write high RSI stocks to second sheet
                    high_rsi = results_df[results_df['RSI'] >= 40].sort_values('RSI', ascending=False)
                    high_rsi.to_excel(writer, sheet_name='High RSI Stocks', index=False)
                    
                    # Get workbook and worksheet objects for formatting
                    workbook = writer.book
                    
                    # Add formats
                    header_format = workbook.add_format({
                        'bold': True,
                        'text_wrap': True,
                        'valign': 'top',
                        'bg_color': '#D9E1F2',
                        'border': 1
                    })
                    
                    # Format each worksheet
                    for worksheet in writer.sheets.values():
                        # Set column widths
                        worksheet.set_column('A:Z', 15)
                        # Apply header format to first row
                        for col_num, value in enumerate(results_df.columns.values):
                            worksheet.write(0, col_num, value, header_format)

                print(f"\n{exchange} analysis complete. Results saved to {excel_file}")
                
                # Print high RSI stocks
                print(f"\n{exchange} Stocks with RSI >= 40:")
                for _, row in high_rsi.iterrows():
                    print(colored(
                        f"{row['Symbol']} ({row['Company_Name']}): RSI = {row['RSI']:.2f}, "
                        f"Price = ₹{row['Latest_Price']:.2f}",
                        'green', attrs=['bold']
                    ))

        print(f"\nDaily analysis complete. Results saved in {self.results_dir}")

if __name__ == "__main__":
    analyzer = StockAnalyzer()
    analyzer.run_daily_analysis()

Loading stock lists...

Analyzing 2061 NSE and 3026 BSE stocks
Debug - AADHARHFC TEMA calculation successfulDebug - AAREYDRUGS TEMA calculation successful
Last TEMA value: 53.19237307689352

Last TEMA value: 407.32195858995556
Debug - AARON TEMA calculation successful
Last TEMA value: 374.99240766978795
Debug - AARTIDRUGS TEMA calculation successful
Last TEMA value: 425.22811947364954
Debug - AARTIIND TEMA calculation successful
Last TEMA value: 408.87861107615913
Debug - 5PAISA TEMA calculation successful
Last TEMA value: 432.46824163447207
Debug - AAKASH TEMA calculation successful
Last TEMA value: 9.858966311899785
Debug - AARTECH TEMA calculation successful
Last TEMA value: 84.51961437259098
Debug - AARTIPHARM TEMA calculation successful
Last TEMA value: 660.7805466538614
Debug - AARTISURF TEMA calculation successful
Last TEMA value: 601.6521096061781
Debug - AARVEEDEN TEMA calculation successful
Last TEMA value: 139.5165881895125
Debug - AARVI TEMA calculation successful
Last TEMA

CAMLIN-RE.NS: Period '1mo' is invalid, must be one of ['1d', '5d']


Debug - CALSOFT TEMA calculation successful
Last TEMA value: 13.255396128373818
Debug - CAMPUS TEMA calculation successful
Last TEMA value: 288.3555096569614
Debug - CAMLINFINE TEMA calculation successful
Last TEMA value: 135.88392705953657
Debug - CANTABIL TEMA calculation successful
Last TEMA value: 296.70040430383506
Debug - CANBK TEMA calculation successful
Last TEMA value: 96.83470903244114
Debug - CANFINHOME TEMA calculation successful
Last TEMA value: 677.3922889649434
Debug - CAMS TEMA calculation successful
Last TEMA value: 4661.747867653727
Debug - CAPITALSFB TEMA calculation successful
Last TEMA value: 299.02882816174605
Debug - CAPACITE TEMA calculation successfulDebug - CAPLIPOINT TEMA calculation successful
Last TEMA value: 2441.9554419790106

Last TEMA value: 408.3548739778907
Debug - CAPTRUST TEMA calculation successful
Last TEMA value: 90.82118226002477
Debug - CAREERP TEMA calculation successful
Last TEMA value: 357.8854191462901
Debug - CARTRADE TEMA calculation succ

NAVKARURB.NS: Period '1mo' is invalid, must be one of ['1d', '5d', 'ytd', 'max']


Debug - MUTHOOTMF TEMA calculation successful
Last TEMA value: 167.13484179168813
Debug - MUTHOOTFIN TEMA calculation successful
Last TEMA value: 2190.1902094251745
Debug - MVGJL TEMA calculation successful
Last TEMA value: 281.13694053834325
Debug - NAGAFERT TEMA calculation successfulDebug - NACLIND TEMA calculation successful
Last TEMA value: 68.56707426458816

Last TEMA value: 8.843038429344674
Debug - NCC TEMA calculation successful
Last TEMA value: 254.1810110951585
Debug - NDTV TEMA calculation successful
Last TEMA value: 154.66007614970647
Debug - PRSMJOHNSN TEMA calculation successful
Last TEMA value: 153.14311487542577
Debug - PRUDENT TEMA calculation successful
Last TEMA value: 2753.7964870659125
Debug - PROZONER TEMA calculation successful
Last TEMA value: 36.88690951927846
Debug - PRUDMOULI TEMA calculation successful
Last TEMA value: 61.869200007669775
Debug - PTC TEMA calculation successfulDebug - PSPPROJECT TEMA calculation successful
Last TEMA value: 667.5373723110317


$504605.BO: possibly delisted; no price data found  (1d 2024-07-04 16:36:36.395690 -> 2025-01-20 16:36:36.395690)


Debug - 505299 TEMA calculation successful
Last TEMA value: 907.265956694109
Debug - 505320 TEMA calculation successful
Last TEMA value: 151.42265842432346
Debug - 505355 TEMA calculation successful
Last TEMA value: 993.6277516748838
Debug - 505324 TEMA calculation successful
Last TEMA value: 20.31194454911279
Debug - 505343 TEMA calculation successful
Last TEMA value: 1.7401130387199988
Debug - 505358 TEMA calculation successful
Last TEMA value: 233.76383415099366
Debug - 505533 TEMA calculation successful
Last TEMA value: 773.3457683697975
Debug - 505368 TEMA calculation successful
Last TEMA value: 370.517085932849
Debug - 505400 TEMA calculation successful
Last TEMA value: 126.4561204343928
Debug - 505537 TEMA calculation successful
Last TEMA value: 125.67394116666074
Debug - 505650 TEMA calculation successful
Last TEMA value: 18.511011281312552
Debug - 505590 TEMA calculation successful
Last TEMA value: 4.676900005958592
Debug - 505681 TEMA calculation successful
Last TEMA value: 6

In [1]:
!pip install --upgrade yfinance